## 0. Import libraries

In [18]:
import pickle
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
from sklearn import preprocessing

from tqdm import tqdm

---
## 1. data preprocessing

### 1) open dataset

In [19]:
file_path = "./dataset_new/"

In [28]:
# 원본 data 부르기
retail_data = pd.read_csv(f"{file_path}Retail data.csv")
industry_data = pd.read_csv(f"{file_path}Industry data.csv")
dealer_hierarchies = pd.read_csv(f"{file_path}Dealer_Hierarchies.csv")

### 2) dataset에 행, 열 통합

- retail, industry data에 지역 정보를 추가한다

In [163]:
# Month가 datetime 형식으로 되어있는데, 이를 필요한 데이터만 얻기 위하여 분리
# datetime 형변환
retail_data["Month"] = pd.to_datetime(retail_data["Month"], format="%Y-%m-%d %H:%M:%S")
industry_data["Month"] = pd.to_datetime(industry_data["Month"], format="%Y-%m-%d %H:%M:%S")

# 열 이름 변경
retail_data.rename(columns={"Month": "Datetime"}, inplace=True)
retail_data.info()

industry_data.rename(columns={"Month": "Datetime"}, inplace=True)
industry_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125226 entries, 0 to 125225
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Datetime      125226 non-null  datetime64[ns]
 1   Sales Total   125226 non-null  int64         
 2   Store         125226 non-null  int64         
 3   Product Type  125226 non-null  object        
 4   Item          125226 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 4.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352894 entries, 0 to 352893
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Datetime       352894 non-null  datetime64[ns]
 1   Industry Size  352894 non-null  int64         
 2   Store          352894 non-null  int64         
 3   Product Type   352894 non-null  object        
 4   Item           352894 non-null  objec

In [164]:
retail_data["Year"] = retail_data["Datetime"].dt.year
retail_data["Month"] = retail_data["Datetime"].dt.month
retail_data["Day"] = retail_data["Datetime"].dt.day
retail_data["Time"] = retail_data["Datetime"].dt.time

industry_data["Year"] = industry_data["Datetime"].dt.year
industry_data["Month"] = industry_data["Datetime"].dt.month
industry_data["Day"] = industry_data["Datetime"].dt.day
industry_data["Time"] = industry_data["Datetime"].dt.time

- day와 time은 필요 없는 값 (1일과 0시만 존재)
- Year, month만 선택해서 datetime을 만들 때 기본 1일 0시로 들어가게 된다

In [167]:
# column 선택
retail_data = retail_data[["Sales Total", "Store", "Product Type", "Item", "Year", "Month", "Datetime"]]
industry_data = industry_data[["Industry Size", "Store", "Product Type", "Item", "Year", "Month", "Datetime"]]

- Store 정보를 추가한다

In [168]:
# retail
region_lv1 = []
region_lv2 = []
urban_rural = []

for i in tqdm(range(len(retail_data))):
    store_num = retail_data.loc[i]["Store"]
    idx = store_num - 1
    region_lv1.append(dealer_hierarchies.loc[idx]["Region Level 1"])
    region_lv2.append(dealer_hierarchies.loc[idx]["Region Level 2"])
    urban_rural.append(dealer_hierarchies.loc[idx]["Urban/Rural"])

retail_data["Region Level 1"] = region_lv1
retail_data["Region Level 2"] = region_lv2
retail_data["Urban/Rural"] = urban_rural

100%|██████████| 125226/125226 [00:51<00:00, 2412.11it/s]


In [169]:
# industry
region_lv1 = []
region_lv2 = []
urban_rural = []

for i in tqdm(range(len(industry_data))):
    store_num = industry_data.loc[i]["Store"]
    idx = store_num - 1
    region_lv1.append(dealer_hierarchies.loc[idx]["Region Level 1"])
    region_lv2.append(dealer_hierarchies.loc[idx]["Region Level 2"])
    urban_rural.append(dealer_hierarchies.loc[idx]["Urban/Rural"])

industry_data["Region Level 1"] = region_lv1
industry_data["Region Level 2"] = region_lv2
industry_data["Urban/Rural"] = urban_rural

100%|██████████| 352894/352894 [02:27<00:00, 2386.15it/s]


- 1-2)에 대한 중간저장과 불러오기를 진행

In [170]:
# *** 나중에 주석 풀어야함 ***
# retail_data.to_csv(f"{file_path}Retail_data_add_region.csv", index=False)
# industry_data.to_csv(f"{file_path}Industry_data_add_region.csv", index=False)

retail_data_ar = pd.read_csv(f"{file_path}Retail_data_add_region.csv")
industry_data_ar = pd.read_csv(f"{file_path}Industry_data_add_region.csv")

### 3) 이상치 제거

- 이상치 (industry < retail)의 경우 모두 0으로 변경한다
- 큰 값으로 맞춰주는 것보다 0으로 변경하는 것의 예측 결과가 더 좋았음

- industry data > retail data 를 확인하기 위해서 store-item-Datetime 열을 생성한다

In [171]:
retail_si = []
for i in tqdm(range(len(retail_data_ar))):
    store = retail_data_ar.loc[i]["Store"]
    item = retail_data_ar.loc[i]["Item"]
    year = retail_data_ar.loc[i]["Year"]
    month = retail_data_ar.loc[i]["Month"]
    retail_si.append(f"{store}-{item}-{year}-{month}")

100%|██████████| 125226/125226 [00:50<00:00, 2502.07it/s]


In [172]:
retail_data_ar["StoreItem"] = retail_si

In [173]:
industry_si = []
for i in tqdm(range(len(industry_data_ar))):
    store = industry_data_ar.loc[i]["Store"]
    item = industry_data_ar.loc[i]["Item"]
    year = industry_data_ar.loc[i]["Year"]
    month = industry_data_ar.loc[i]["Month"]
    industry_si.append(f"{store}-{item}-{year}-{month}")

100%|██████████| 352894/352894 [02:20<00:00, 2513.04it/s]


In [174]:
industry_data_ar["StoreItem"] = industry_si

- 잘못된 인덱스 저장, 삭제

In [175]:
retail_err_idx = []
industry_err_idx = []

In [176]:
for i in tqdm(range(len(retail_data_ar))):
    storeitem = retail_data_ar.loc[i]["StoreItem"]
    same_item_df = industry_data_ar[industry_data_ar["StoreItem"] == storeitem]
    if len(same_item_df) == 0:
        retail_err_idx.append(i)
        continue

    if same_item_df.iloc[0]["Industry Size"] < retail_data_ar.loc[i]["Sales Total"]:  # Sales total이 더 크면 error
        retail_err_idx.append(i)
        industry_err_idx.append(same_item_df.index[0])

100%|██████████| 125226/125226 [47:22<00:00, 44.06it/s]


In [177]:
print(len(retail_err_idx))
print(len(industry_err_idx))

23401
4205


In [178]:
# 잘못된 인덱스 파일 pkl로 저장

# ***
# with open('retail_err_idx.pkl','wb') as f:
#     pickle.dump(retail_err_idx,f)
# with open('industry_err_idx.pkl','wb') as f:
#     pickle.dump(industry_err_idx,f)

with open("retail_err_idx.pkl", "rb") as f:
    retail_err_idx = pickle.load(f)
with open("industry_err_idx.pkl", "rb") as f:
    industry_err_idx = pickle.load(f)

- label 을 없앤다

In [179]:
retail_data_ar_delidx = retail_data_ar.drop(retail_err_idx).reset_index(drop=True)
industry_data_ar_delidx = industry_data_ar.drop(industry_err_idx).reset_index(drop=True)

retail_data_ar_delidx["Store"] = retail_data_ar_delidx["Store"].astype(str)
industry_data_ar_delidx["Store"] = industry_data_ar_delidx["Store"].astype(str)

In [181]:
retail_data_ar_delidx = retail_data_ar_delidx[
    ["Sales Total", "Store", "Product Type", "Item", "Year", "Month", "Datetime", "Region Level 1", "Region Level 2", "Urban/Rural"]
]
industry_data_ar_delidx = industry_data_ar_delidx[
    ["Industry Size", "Store", "Product Type", "Item", "Year", "Month", "Datetime", "Region Level 1", "Region Level 2", "Urban/Rural"]
]

In [182]:
retail_data_ar_delidx.head(3)

,Sales Total,Store,Product Type,Item,Year,Month,Datetime,Region Level 1,Region Level 2,Urban/Rural
0,1,1,Electronics,Phone Charger,2021,2,2021-02-01,Eisenstadt-Umgebung,GWR,Urban
1,5,1,Grocery,Shrimp,2021,2,2021-02-01,Eisenstadt-Umgebung,GWR,Urban
2,2,1,Grocery,Noodles,2021,2,2021-02-01,Eisenstadt-Umgebung,GWR,Urban


- owner 정보 추가

In [183]:
retail_owner = []
industry_owner = []

for i in tqdm(range(len(retail_data_ar_delidx))):
    store = int(retail_data_ar_delidx.loc[i]["Store"])
    owner = dealer_hierarchies.loc[store - 1]["Store Owner"]
    retail_owner.append(owner)

for i in tqdm(range(len(industry_data_ar_delidx))):
    store = int(industry_data_ar_delidx.loc[i]["Store"])
    owner = dealer_hierarchies.loc[store - 1]["Store Owner"]
    industry_owner.append(owner)

100%|██████████| 348689/348689 [01:07<00:00, 5199.44it/s]


In [184]:
retail_data_ar_delidx["Owner"] = retail_owner
industry_data_ar_delidx["Owner"] = industry_owner

- 1-3)에 대한 중간저장과 불러오기를 진행

In [185]:
# 3)에 대한 중간저장과 불러오기를 진행
# *** 나중에 주석 풀어야함 ***
# retail_data_ar_delidx.to_csv(f"{file_path}retail_all.csv", index=False)
# industry_data_ar_delidx.to_csv(f"{file_path}industry_all.csv", index=False)

retail_data = pd.read_csv(f"{file_path}retail_all.csv")
industry_data = pd.read_csv(f"{file_path}industry_all.csv")

### 4) 빈 데이터 채우기

- 기존 industry_data는 348689행이지만, 0인 정보들 고려 위해서 전체 행에 해당하는 열 만들어줌
- industry_data_add0: 1,005,372행 가진 data

In [186]:
datetime_list = list(dict.fromkeys(list(industry_data["Datetime"])))
store_list = [i for i in range(1, 643)]  # 1~642

In [187]:
product_dict = {
    "Power Cord": "Electronics",
    "Phone Charger": "Electronics",
    "Ear Buds": "Electronics",
    "Mouse": "Electronics",
    "Keyboard": "Electronics",
    "Milk": "Grocery",
    "Eggs": "Grocery",
    "Cereal": "Grocery",
    "Shrimp": "Grocery",
    "Noodles": "Grocery",
    "Steak": "Grocery",
    "King Crab": "Grocery",
    "Tape": "Home Goods",
    "Glue": "Home Goods",
    "Nails": "Home Goods",
    "Bracket": "Home Goods",
    "Brush": "Home Goods",
    "Paint": "Home Goods",
}

In [188]:
# 추가할 dataframe
new_df = pd.DataFrame(
    [], columns=["Industry Size", "Store", "Product Type", "Item", "Year", "Month", "Datetime", "Region Level 1", "Region Level 2", "Urban/Rural", "Owner"]
)

In [189]:
from re import S


def append_new_sentence(new_df, product, d, s):
    new_df.loc[len(new_df)] = [
        0,  # industry size
        s,  # store
        product_dict[product],  # item
        product,  # product type
        int(d[:4]),  # year
        int(d[5:7]),  # month
        d,  # datetime
        dealer_hierarchies.loc[s - 1]["Region Level 1"],
        dealer_hierarchies.loc[s - 1]["Region Level 2"],
        dealer_hierarchies.loc[s - 1]["Urban/Rural"],
        dealer_hierarchies.loc[s - 1]["Store Owner"],
    ]

In [190]:
# 재고 0인 행 목록에 추가
for d in tqdm(datetime_list):
    for product in list(product_dict.keys()):  # key
        for s in store_list:
            datetime_df = industry_data[(industry_data["Datetime"] == d) & (industry_data["Store"] == s)]  # 해당 datetime의 df
            if len(datetime_df[datetime_df["Item"] == product]) == 0:
                append_new_sentence(new_df, product, d, s)

 14%|█▍        | 12/87 [1:05:10<6:47:22, 325.90s/it]


KeyboardInterrupt: 

In [ ]:
# 추가
industry_data_1_add0 = industry_data.append(new_df, ignore_index=True)

# 정렬 # 1040040 개여야함
industry_data_1_add0.sort_values(["Datetime", "Item", "Store"], inplace=True)
industry_data_1_add0.reset_index(drop=True, inplace=True)

- 1-4)에 대한 중간저장과 불러오기를 진행

In [ ]:
# ***
# industry_data_1_add0.to_csv('./industry_data_add0.csv',index=False)
industry_data_1_add0 = pd.read_csv("./dataset_new/industry_data_add0.csv")

---
## 2. Xgboost 이용한 예측 모델 생성

### 1) Data 준비

- 1에서 생성한 파일 오픈

In [131]:
industry_data_1_add0 = pd.read_csv("./dataset_new/industry_data_add0.csv")

In [144]:
industry_data_1_add0

,Industry Size,Store,Product Type,Item,Year,Month,Datetime,Region Level 1,Region Level 2,Urban/Rural,Owner,quarter
0,0,1,Home Goods,Bracket,2015,1,2015-01-01,Eisenstadt-Umgebung,GWR,Urban,1,1
1,0,1,Home Goods,Brush,2015,1,2015-01-01,Eisenstadt-Umgebung,GWR,Urban,1,1
2,0,1,Grocery,Cereal,2015,1,2015-01-01,Eisenstadt-Umgebung,GWR,Urban,1,1
3,0,1,Electronics,Ear Buds,2015,1,2015-01-01,Eisenstadt-Umgebung,GWR,Urban,1,1
4,0,1,Grocery,Eggs,2015,1,2015-01-01,Eisenstadt-Umgebung,GWR,Urban,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1040035,0,642,Electronics,Phone Charger,2022,06,2022-06-01,Koillismaa,KTR,Urban,74,1
1040036,0,642,Electronics,Power Cord,2022,06,2022-06-01,Koillismaa,KTR,Urban,74,1
1040037,0,642,Grocery,Shrimp,2022,06,2022-06-01,Koillismaa,KTR,Urban,74,1
1040038,0,642,Grocery,Steak,2022,06,2022-06-01,Koillismaa,KTR,Urban,74,1


- 4,5,6월 예측을 위한 행 생성

In [133]:
def add_date(date, original_df):  # date: 'YYYY-MM-DD'형태
    data_predict = industry_data_1_add0[industry_data_1_add0["Datetime"] == "2022-03-01"].reset_index(drop=True)
    data_predict["Industry Size"] = [0] * len(data_predict)
    data_predict["Datetime"] = [date] * len(data_predict)
    data_predict["Year"] = [date[:4]] * len(data_predict)
    data_predict["Month"] = [date[5:7]] * len(data_predict)

    return pd.concat([original_df, data_predict], ignore_index=True)

In [134]:
industry_data_1_add0 = add_date("2022-04-01", industry_data_1_add0)
industry_data_1_add0 = add_date("2022-05-01", industry_data_1_add0)
industry_data_1_add0 = add_date("2022-06-01", industry_data_1_add0)

- 제출 형식에 맞게 정렬

In [136]:
industry_data_1_add0.sort_values(["Datetime", "Store", "Item"], inplace=True)
industry_data_1_add0.reset_index(drop=True, inplace=True)

- Label encoding

In [138]:
label = preprocessing.LabelEncoder()
industry_data_encoded = pd.DataFrame()

industry_data_1_col = ["Product Type", "Item", "Region Level 1", "Region Level 2", "Urban/Rural"]
for i in industry_data_1_add0.columns:
    if i in industry_data_1_col:
        industry_data_encoded[i] = label.fit_transform(industry_data_1_add0[i])
    else:
        industry_data_encoded[i] = industry_data_1_add0[i]

### 2) 모델 생성 및 예측

- 예측할 구간 선택

In [139]:
set_dt_list = ["2022-04-01", "2022-05-01", "2022-06-01"]

- 모델 생성
    - 2015-01-01 ~ 2022-03-01 data --> 2022-04-01 data 예측	
    - 2015-01-01 ~ 2022-04-01 data --> 2022-05-01 data 예측	
    - 2015-01-01 ~ 2022-05-01 data --> 2022-06-01 data 예측	

In [140]:
for set_dt in set_dt_list:
    # train에 사용할 data와 예측할 data 분리
    train_1 = industry_data_encoded[industry_data_encoded["Datetime"] < set_dt]
    val_1 = industry_data_encoded[(industry_data_encoded["Datetime"] >= set_dt)]

    # Feature와 Target 분리
    target = "Industry Size"
    features = train_1.drop([target, "Datetime", "quarter", "Region Level 1", "Region Level 2"], axis=1).columns

    X_train = train_1[features]
    y_train = train_1[target]
    X_val = val_1[features]
    y_val = val_1[target]

    print(X_train.shape, y_train.shape)
    print(X_val.shape, y_val.shape)

    # 파이프라인 생성 및 학습
    xgb_basic = make_pipeline(
        SimpleImputer(),
        RobustScaler(),
        XGBRegressor(
            random_state=1,
            n_jobs=-1,
            objective="reg:squarederror",
            max_depth=5,
            learning_rate=0.2,
        ),
    )

    xgb_basic.fit(X_train, y_train)

    # 예측
    y_pred = xgb_basic.predict(X_val)

    # prediction값 반올림
    y_pred_r = [round(x) if round(x) >= 0 else 0 for x in y_pred]

    # 결과 확인
    print("mae:  ", mae(y_val, y_pred_r))
    print("rmse: ", np.sqrt(mse(y_val, y_pred_r)))

    # prediction 값 추가
    newlist = list(industry_data_encoded[industry_data_encoded["Datetime"] < set_dt]["Industry Size"]) + y_pred_r
    industry_data_encoded["Industry Size"] = newlist

(1005372, 7) (1005372,)
(34668, 7) (34668,)
mae:   1.1522441444559826
rmse:  1.7096735745150142
(1016928, 7) (1016928,)
(23112, 7) (23112,)
mae:   0.11673589477327795
rmse:  0.341919530064906
(1028484, 7) (1028484,)
(11556, 7) (11556,)
mae:   0.12287988923502942
rmse:  0.3537369117011909


In [141]:
result = pd.DataFrame([])
result["predicted"] = industry_data_encoded[industry_data_encoded["Datetime"] >= set_dt_list[0]]["Industry Size"].reset_index(drop=True)

- 4, 5, 6월 데이터가 한 줄로 예측되어있으므로 4+5+6월을 한 값을 저장한다

In [142]:
sum_456 = result[:11556].copy()
for i in range(len(sum_456)):
    m_5 = result.loc[i + 11556]["predicted"]
    m_6 = result.loc[i + 11556 + 11556]["predicted"]
    sum_456.loc[i, "predicted"] = sum_456.loc[i, "predicted"] + m_5 + m_6

In [143]:
# 예측된 전체 데이터 개수 확인
sum_456["predicted"].sum()

39946

---
## 3. 제출 파일 생성

### 1) 제출 양식 생성

- 제출 양식에 맞는 파일을 생성하기 위하여 `가게-아이템 목록-빈값` dataframe을 생성한다

In [46]:
profit_per_product = pd.read_csv(f"{file_path}Profit per Product.csv")
product_list = sorted(list(profit_per_product["Item"])[:18])  # item:18개

In [90]:
make_template_store = []
make_template_item = product_list * 642
for i in range(1, 643):
    append_store = [i] * len(product_list)
    make_template_store.extend(append_store)

submission_template = pd.DataFrame([])
submission_template["Store"] = make_template_store
submission_template["Item"] = make_template_item
submission_template["Allocation"] = [0] * 11556

### 2) 5k 제출 파일 생성

In [91]:
submission_5k = submission_template.copy()
submission_5k["Allocation"] = sum_456["predicted"]

- 예측된 값이 39946개이므로 이를 5000개로 줄여야 한다.
    - 가격 낮은 순 `["Nails", "Milk", "Tape", "Bracket", "Eggs", "Cereal", "Brush", "Glue", "Power Cord", "Noodles"]`까지 모두 삭제한다.
    - 많이 예측된 항목일 경우 실제로도 많이 할당될 가능성이 높으므로 더 작은 수로 나누어 많이 할당되게끔 한다.
    - 최대한 5000과 비슷한 수가 나오게끔 만든다.

In [92]:
check1 = []
divided = []
for i in range(len(submission_5k)):
    now_num = submission_5k.loc[i]["Allocation"]

    # 0으로 만들 값 체크
    zerolist = ["Nails", "Milk", "Tape", "Bracket", "Eggs", "Cereal", "Brush", "Glue", "Power Cord", "Noodles"]  # ,'Ear Buds']
    if submission_5k.loc[i]["Item"] in zerolist:
        divided.append(0)
        continue  # 아래 실행 x

    # 1보다 큰지 체크
    if now_num >= 1:
        check1.append(1)
    else:
        check1.append(0)

    # 값 체크
    if now_num > 30:
        divided.append(round(now_num / 5))
    elif now_num > 20:
        divided.append(round(now_num / 6.5))
    elif now_num > 10:
        divided.append(round(now_num / 7.5))

    elif now_num >= 1:
        divided.append(1)
    else:
        divided.append(0)

In [93]:
sum(divided)

5006

- 위의 코드를 수행하니 5006개가 나와서, 6개를 더 줄여야 한다.

In [94]:
submission_5k["Allocation"] = divided

- 가격 낮은 순 Nails ~ Noodles 다음으로 가격이 낮은 Ear Buds에 대해 많이 할당된 것 6개를 삭제

In [95]:
submission_5k[(submission_5k["Item"] == "Ear Buds") & (submission_5k["Allocation"] > 5)]

,Store,Item,Allocation
1659,93,Ear Buds,7
10947,609,Ear Buds,7


In [145]:
submission_5k.loc[1659, "Allocation"] = 4
submission_5k.loc[10947, "Allocation"] = 4

In [146]:
submission_5k["Allocation"].sum()

5000

- 5000개가 맞춰졌으므로 저장한다.

In [83]:
submission_5k.to_csv(f"JumpingCat5k.csv", index=False)

### 3) 10k 제출 파일 생성

In [84]:
submission_10k = submission_template.copy()
submission_10k["Allocation"] = sum_456["predicted"]

- 예측된 값이 39946개이므로 이를 5000개로 줄여야 한다.
    - 가격 낮은 순 `["Nails", "Milk", "Tape", "Bracket", "Eggs"]`까지 모두 삭제한다.
    - 많이 예측된 항목일 경우 실제로도 많이 할당될 가능성이 높으므로 더 작은 수로 나누어 많이 할당되게끔 한다.
    - 추후 가장 ABC score가 높은 Keyboards를 더 할당하기 위하여 Keyboard의 값을 저장해둔다.
    - 최대한 10000과 비슷한 수가 나오게끔 만든다.

In [98]:
check1 = []
divided = []
keyboard = {}
for i in range(len(submission_10k)):
    now_num = submission_10k.loc[i]["Allocation"]

    # 0으로 만들 값 체크
    zerolist = ["Nails", "Milk", "Tape", "Bracket", "Eggs"]

    if submission_10k.loc[i]["Item"] in zerolist:
        divided.append(0)
        continue  # 아래 실행 x

    if submission_10k.loc[i]["Item"] == "Keyboard":
        keyboard[i] = now_num

    # 1보다 큰지 체크
    if now_num >= 1:
        check1.append(1)
    else:
        check1.append(0)

    # 값 체크
    if now_num > 30:
        divided.append(round(now_num / 2))
    elif now_num > 20:
        divided.append(round(now_num / 2.3))
    elif now_num > 10:
        divided.append(round(now_num / 3.3))
    elif now_num > 5:
        divided.append(round(now_num / 4.5))
    elif now_num >= 1:
        divided.append(1)
    else:
        divided.append(0)

In [99]:
sum(divided)

9904

- 위의 코드를 수행하니 9904개가 나와서, 96개를 더 늘려야 한다.
- 가장 ABC 점수가 높은 Keyboard를 더 늘릴 예정이다.
    - 확인 결과 Keyboard가 0개 할당 된 것은 없으므로, 1개 할당된 것 중 96개 index에 대하여 1개씩 더 할당해준다.

In [102]:
submission_10k["Allocation"] = divided

In [105]:
idxs = submission_10k[(submission_10k["Item"] == "Keyboard") & (submission_10k["Allocation"] == 1)].index
len(idxs)

261

- Keyboard가 1개 할당된 것은 261개이므로 일부 index에 대해서만 하나씩 더 할당해준다.

In [106]:
for i in idxs[-96:]:  # 마지막부터 96개
    submission_10k.loc[i, "Allocation"] = 2

In [107]:
submission_10k["Allocation"].sum()

10000

- 10000개가 맞춰졌으므로 저장한다.

In [109]:
submission_10k.to_csv(f"JumpingCat10k.csv", index=False)